In [1]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.remove_shortcut('up');
Jupyter.keyboard_manager.command_shortcuts.remove_shortcut('down');

<IPython.core.display.Javascript object>

In [2]:
from posixpath import lexists
import math
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.cm as cm
#from google.colab import files
import csv

In [3]:
#INPUTS
run_kinetics_plotter = False              #plots all scans of the WELL_NAMES provided below
run_scan_plotter = False                  #plots all wells of the SCAN_NAMES provided below
run_all_data_plotter = False              #plots all data together
run_abs_vs_time_plotter = False           #plots the absorbance(s) of the provided WAVELENGTHS_TO_PLOT(s) for the WELL_NAMES provided below
run_lambda_max_plotter = False            #plots the absorbance of the lambda max within the provided WAVELENGTHS_RANGE for the WELL_NAMES provided below
run_lambda_max_output = True              #outputs the lamba max wavelength and absorbance at that wavelength within the provided WAVELENGTHS_RANGE for the WELL_NAMES provided below
run_specific_wells_scans_plotter = False  #plots the WELL_NAMES within the SCAN_NAMES provided below
run_kinetics_summary_plotter = False      #plots all scans on all wells in individual subplots arranged based on their well position

experiment_name = "CNH_040"
well_names = ["A02","B02","D02","E02"]
scan_names = ["scan_024"]
wavelength_to_plot = 550
wavelength_range = (350, 950)
save_or_not = False
subtract_blanks_or_not = True
blank_well = "A01"
initial_time_as_last_reagent_added = True
presentation_mode = False #reduces detail and makes thicker lines for kinetics_summary_plotter

#provide the name of the experiment, as spelled in the original sheet, in quotes. This is how the code will know where to find the data.
#ex: experiment_name = "CNH_test"
#provide the wells you want to plot (for the plotting methods that go by well). Provide each well name in quotes, and if you want multiple wells on the same plot include them together in brackets
#ex: well_names = ["A05", "G05"], "G05", "H05"
#provide the scans you want to plot (for the plotting methods that go by scan). Provide each scan name in quotes, and if you want multiple wells on the same plot include them together in brackets
#ex: scan_names = ["scan_1", "scan_3"], "scan_4"
#provide the wavelengths you want plotted as integers, separated by commas
#ex: wavelength_to_plot = 305, 400
#provide the wavelength range you want to find lambda max in, as integers in paranthesis separated by a comma with the lower bound first and upper bound second
#ex: wavelength_range = (450, 800)
#decide whether you want to save the plots with True or False



#THINGS TO CHANGE ABOUT STANDARD DATAFRAME
#add column to far left for experiment name (so we can compare multiple experiments in the future by merging the dstaframes)
#add column with reaction name (probably leave column with reaction well in there)
#add columns for time of last reagent added
#add columns for all reagent concentrations at the time of scan (so we can include this information in the plots)


#add option for multiple experiments (probably by adding exp name to scan/well names)
#add option to use differences among a specified reagent rather than well names
#add option to use reagent concentration rather than well name in labels

#add error message when calling wells/scans that don't exist for specific_wells_scans_plotter
#add plotter that does time ranges
#add option to work on local desktop rather than drive

#make GUI to make this more user-freindly
#clean up plate summary plot (and implement presentation modes)

#Assumption: wavelengths are initially found in the top non-title row
#Assumption: the first data is always immediately to the right of the "time" column




def load_df(path, subtract_blanks_or_not, blank_well, initial_time_as_last_reagent_added):
  #loads the dataframe
  df = pd.read_csv(path)

  #produces a list of the wavelengths as integers to be returned for future use in the code
  wavelengths = df.columns.tolist()
  wavelengths = wavelengths[wavelengths.index("time")+1:]
  wavelengths = list(map(int, wavelengths))
  for i in wavelengths:
    df = df.rename({str(i):i}, axis='columns')

  #createsa new row with the wavelength data (really all of the column names) and renames a couple of other cells
  df = df.rename({"Unnamed: 0":"Scan_Number" }, axis='columns')
  df.loc[0] = df.columns.tolist()

  #renames scan id's in numerical order they were done
  time_names = list(set(df.loc[1:,"time"]))
  time_names.sort()
  df = df.rename({"scan id":"original_scan_id" }, axis='columns')
  scan_id_dictionary = {'time':'time'}
  for i in range(0,len(time_names)):
    scan_name = "scan_" + str(i).zfill(3)
    scan_id_dictionary[time_names[i]] = scan_name
  scan_id_list = []
  for i in df['time']:
    scan_id_list = scan_id_list + [scan_id_dictionary[i]]
  df.loc[:,'scan id'] = scan_id_list

  #renames a couple of other cells
  df["scan id"][0]="Wavelength"
  df["well"][0]="Wavelength"

  #produces lists of the wells and scans to be returned for future use in the code
  wells = list(set(df.loc[1:,"well"]))
  wells.sort()
  scans = list(set(df.loc[1:,"scan id"]))
  scans.sort()
  print(scans)

  #sets a multindex with the scan id as the top level (0) index and the well as the next level (1)
  df = df.set_index(["scan id", "well"]).sort_index()
  

  #adds a column with the delta times between scans for each well
  #to make the math easier this is done by well, so the df is reorganized by well, the column added, and then it is reorganized back
  list_of_time_floats = []
  for i in wells:
    data = df.loc[pd.IndexSlice[:, ["Wavelength", i], :]]
    if initial_time_as_last_reagent_added:
      initial_time = datetime.strptime(data.iloc[-1,data.columns.get_loc("time of last reagent added")], "%Y-%m-%d %H:%M:%S:%f")
    else:
      initial_time = datetime.strptime(data.iloc[1,data.columns.get_loc("time")], "%Y-%m-%d %H:%M:%S")
    number_of_scans = data.shape[0]
    for j in range(1,number_of_scans):
      scan_time = datetime.strptime(data.iloc[j,data.columns.get_loc("time")], "%Y-%m-%d %H:%M:%S")
      time_difference = scan_time - initial_time
      list_of_time_floats = list_of_time_floats + [time_difference.total_seconds()/60]

  df = df.swaplevel(0,1).sort_index()
  list_of_time_floats = list_of_time_floats + ["NaN"]
  df.insert(df.columns.get_loc("time"), "Time_Since_Last_Scan",list_of_time_floats)
  df = df.swaplevel(0,1).sort_index()
  

  #subtracts the blanks if requested
  if subtract_blanks_or_not:
    if blank_well not in wells: raise Exception("The provided blank well (" + str(blank_well) + ") was never scanned so blanks cannot be subtracted as requested")
    unsubtracted_df = df.copy()
    past_blank_time = datetime.strptime("2001-01-01 12:00:00", "%Y-%m-%d %H:%M:%S")
    future_blank_time = datetime.strptime("2100-01-01 12:00:00", "%Y-%m-%d %H:%M:%S")
    list_of_blank_scans = []
    for i in scans:
      data = df.loc[pd.IndexSlice[["Wavelength"] + [i]],:, :]
      #checks to see if there is a blank in the current scan, and if so adds that scan to the list and sets the past_blank_time to that time for future scans
      if blank_well in list(data.index.get_level_values(1))[1:]:
        past_blank_time = datetime.strptime(data.iloc[1,data.columns.get_loc("time")], "%Y-%m-%d %H:%M:%S") 
        past_blank_scan = i
        list_of_blank_scans = list_of_blank_scans + [i]
      #if there isn't a blank in this scan, try using an earlier/later blank
      else:
        time_of_scan_missing_blank = datetime.strptime(data.iloc[1,data.columns.get_loc("time")], "%Y-%m-%d %H:%M:%S")
        future_blank_time = datetime.strptime("2100-01-01 12:00:00", "%Y-%m-%d %H:%M:%S")
        #checking for future blanks
        for j in scans[scans.index(i):]:
          data = df.loc[pd.IndexSlice[["Wavelength"] + [j]],:, :]
          if blank_well in list(data.index.get_level_values(1))[1:]:
            future_blank_time = datetime.strptime(data.iloc[1,data.columns.get_loc("time")], "%Y-%m-%d %H:%M:%S")
            future_blank_scan = j
            break
        #decides whether the past or future scan is closer in time and uses that one for the blank   
        if future_blank_time - time_of_scan_missing_blank < time_of_scan_missing_blank - past_blank_time:
          list_of_blank_scans = list_of_blank_scans + [future_blank_scan]
        else:
          list_of_blank_scans = list_of_blank_scans + [past_blank_scan]
    
    #now that we have a list of blank scans we go through and subtract them from every well in every scan in the dataframe
    for i in range(len(scans)):
      data = df.loc[pd.IndexSlice[["Wavelength"] + [scans[i]]],:, :]
      list_of_blank_data = np.array(unsubtracted_df.loc[(list_of_blank_scans[i],blank_well),wavelengths])
      
      for j in list(data.index.get_level_values(1))[1:]:
        list_of_data = np.array(df.loc[(scans[i],j),wavelengths])
        subtracted_data = np.subtract(list_of_data, list_of_blank_data)
        df.loc[(scans[i],j),wavelengths] = subtracted_data

    del unsubtracted_df, list_of_blank_data


  return df, wavelengths, scans, wells


#prepares the graph environment. does not change the data
def plot_prepper(wavelengths, plot_title):
  number_wavelength_tick_marks = 8
  number_y_tick_marks = 5
  y_max = 1
  y_min = 0
  y_tick_marks = [i/number_y_tick_marks for i in range(0,number_y_tick_marks+1,1)]
  y_tick_marks = [y_max*i for i in y_tick_marks]

  (wavelength_min, wavelength_max) = (wavelengths[0], wavelengths[-1])
  wavelength_range = wavelength_max - wavelength_min
  wavelength_tick_mark_separation = wavelength_range / (number_wavelength_tick_marks-1)
  wavelength_tick_marks = [wavelength_min]
  for i in range(number_wavelength_tick_marks-1):
    wavelength_tick_marks = wavelength_tick_marks + [wavelength_tick_marks[-1]+wavelength_tick_mark_separation]

  plt.figure(num=None, figsize=(4, 4),dpi=300, facecolor='w', edgecolor='k')
  plt.legend(loc="upper right",frameon = False, prop={"size":7},labelspacing = 0.5)
  plt.rc('axes', linewidth = 2)
  plt.xlabel('Wavelength (nm)',fontsize = 16)
  plt.ylabel('Absorbance (a.u.)', fontsize = 16)
  plt.tick_params(axis = "both", width = 2)
  #plt.tick_params(axis = "both", width = 2)
  plt.xticks(wavelength_tick_marks)
  plt.yticks(y_tick_marks)
  plt.axis([wavelength_min, wavelength_max, y_min , y_max])
  plt.xticks(fontsize = 14)
  plt.yticks(fontsize = 14)
  plt.title(str(plot_title), fontsize = 16, pad = 20)



def plot_prepper2_part1(plot_title):
  y_max = 1
  y_min = 0
  time_min = 0
  time_max = 10
  plt.figure(num=None, figsize=(4, 4),dpi=300, facecolor='w', edgecolor='k')
  plt.legend(loc="upper right",frameon = False, prop={"size":7},labelspacing = 0.5)
  plt.rc('axes', linewidth = 2)
  plt.xlabel('Time (minutes)',fontsize = 16)

  plt.axis([time_min, time_max, y_min , y_max])
  plt.title(str(plot_title), fontsize = 16, pad = 20)


def plot_prepper2_part2(total_time, y_min, y_max, y_title):


  number_y_tick_marks = 5
  y_tick_marks = [i/number_y_tick_marks for i in range(0,number_y_tick_marks+1,1)]
  y_tick_marks = [(y_max-y_min)*i+y_min for i in y_tick_marks]

  number_time_tick_marks = 5
  time_min = 0

  #rounds the x-axis maximum up to the nearest 10 minute, i.e. both 11 min and 19 min round to 20 min
  time_max = (int(math.ceil(total_time/10)))*10
  x_tick_marks = [i/number_time_tick_marks for i in range(0,number_time_tick_marks+1,1)]
  x_tick_marks = [time_max*i for i in x_tick_marks]
  plt.ylabel(y_title, fontsize = 16)
  plt.axis([time_min, time_max, y_min , y_max])
  plt.tick_params(axis = "both", width = 2)
  plt.xticks(x_tick_marks)
  plt.xticks(fontsize = 14)
  plt.yticks(y_tick_marks)
  plt.yticks(fontsize = 14)


def kinetics_plotter(df, wavelengths, well_names, wells, experiment_name, save_or_not, save_path):
  for i in well_names:
    if i not in wells: raise Exception("The provided well (" + str(i) + ") was never scanned so it cannot be plotted")
  plot_title = ", ".join(well_names) + " Kinetics"
  plot_prepper(wavelengths, plot_title)
  list_of_time_strings = []

  data = df.loc[pd.IndexSlice[:, ["Wavelength"] + well_names], :]
  number_of_scans = data.shape[0]
  colors = list(cm.rainbow(np.linspace(0, 1, number_of_scans)))
  
  initial_time = datetime.strptime(data.iloc[1,data.columns.get_loc("time")], "%Y-%m-%d %H:%M:%S")
  list_of_time_floats = list(data["Time_Since_Last_Scan"])[1:]
  list_of_temperatures = list(data["temp"])[1:]

  for i in range(1,number_of_scans):
    #creates string labels from the list of times (adds the well name if multiple wells are on the same plot)
    if len(well_names) == 1:
      list_of_time_strings = list_of_time_strings + [str(round(list_of_time_floats[i-1],1)) + " minutes"]
    else: 
      list_of_time_strings = list_of_time_strings + [str(data.index.tolist()[i][1]) + " " + str(round(list_of_time_floats[i-1],1)) + " minutes"]
    

    x_data = data.iloc[data.index.get_loc("Wavelength"),df.columns.get_loc("time")+1:].transpose()
    y_data = data.iloc[i,df.columns.get_loc("time")+1:]

    plt.plot(x_data, y_data,color=tuple(colors[i-1]))

  patches = [mpatches.Patch(color=color, label=list_of_time_strings) for list_of_time_strings, color in zip(list_of_time_strings, colors)]
  plt.legend(patches, list_of_time_strings, loc='upper right', frameon=False,prop={'size':8})
  
  max_temp = max(list_of_temperatures)
  min_temp = min(list_of_temperatures)
  metadata_text = experiment_name + "\n" + "Range of temperatures during this experiment: " + str(min_temp) + " - " + str(max_temp) + " C \n The first scan for this experiment occurred at: " + str(initial_time)
  plt.gcf().text(0.5, -0.2, metadata_text, fontsize = 8, horizontalalignment="center")

  save_path = save_path + experiment_name + "_" + "_".join(well_names) + "_kinetics.png"
  if save_or_not: plt.savefig(save_path, bbox_inches='tight')





def scan_plotter(df, wavelengths, scan_names, scans, experiment_name, save_or_not, save_path):
  for i in scan_names:
    if i not in scans: raise Exception("The provided scans (" + str(i) + ") does not exist so it cannot be plotted")
  
  plot_title = "Data from " + ", ".join(scan_names)
  plot_prepper(wavelengths, plot_title)

  data = df.loc[pd.IndexSlice[["Wavelength"] + scan_names],:, :]
  list_of_wells = []
  list_of_temperatures = []
  number_of_wells = data.shape[0]
  colors = list(cm.rainbow(np.linspace(0, 1, number_of_wells)))
  initial_time = datetime.strptime(data.iloc[1,data.columns.get_loc("time")], "%Y-%m-%d %H:%M:%S")

  for i in range(1,number_of_wells):
    if len(scan_names) == 1:
      list_of_wells = list_of_wells + [data.index.tolist()[i][1]]
    else: 
      list_of_wells = list_of_wells + [data.index.tolist()[i][0] + " " + data.index.tolist()[i][1]]
    
    list_of_temperatures = list_of_temperatures + [data.iloc[i,data.columns.get_loc("temp")]]

    x_data = data.iloc[data.index.get_loc("Wavelength"),df.columns.get_loc("time")+1:].transpose()
    y_data = data.iloc[i,df.columns.get_loc("time")+1:]

    plt.plot(x_data, y_data,color=tuple(colors[i-1]))

  patches = [mpatches.Patch(color=color, label=list_of_wells) for list_of_wells, color in zip(list_of_wells, colors)]
  plt.legend(patches, list_of_wells, loc='upper right', frameon=False,prop={'size':8})

  max_temp = max(list_of_temperatures)
  min_temp = min(list_of_temperatures)
  metadata_text = experiment_name + "\n" + "Range of temperatures during this experiment: " + str(min_temp) + " - " + str(max_temp) + " C \n The first scan for this experiment occurred at: " + str(initial_time)
  plt.gcf().text(0.5, -0.2, metadata_text, fontsize = 8, horizontalalignment="center")

  save_path = save_path + experiment_name + "_" + "_".join(scan_names) + ".png"
  if save_or_not: plt.savefig(save_path, bbox_inches='tight')


def specific_wells_scans_plotter(df,wavelengths, well_names, scan_names, experiment_name, save_or_not, save_path):
  plot_title = "Data from " + ", ".join(scan_names) + " " + ", ".join(well_names)
  plot_prepper(wavelengths, plot_title)
  data = df.loc[pd.IndexSlice[["Wavelength"] + scan_names, ["Wavelength"] + well_names], :]
  list_of_labels = []
  list_of_temperatures = []
  number_of_lines_to_plot = data.shape[0]
  colors = list(cm.rainbow(np.linspace(0, 1, number_of_lines_to_plot)))
  initial_time = datetime.strptime(data.iloc[1,data.columns.get_loc("time")], "%Y-%m-%d %H:%M:%S")

  for i in range(1,number_of_lines_to_plot):
    list_of_labels = list_of_labels + [data.index.tolist()[i][0] + " " + data.index.tolist()[i][1]]  
    list_of_temperatures = list_of_temperatures + [data.iloc[i,data.columns.get_loc("temp")]]

    x_data = data.iloc[data.index.get_loc("Wavelength"),df.columns.get_loc("time")+1:].transpose()
    y_data = data.iloc[i,df.columns.get_loc("time")+1:]

    plt.plot(x_data, y_data,color=tuple(colors[i-1]))

  patches = [mpatches.Patch(color=color, label=list_of_labels) for list_of_labels, color in zip(list_of_labels, colors)]
  plt.legend(patches, list_of_labels, loc='upper right', frameon=False,prop={'size':8})

  max_temp = max(list_of_temperatures)
  min_temp = min(list_of_temperatures)
  metadata_text = experiment_name + "\n" + "Range of temperatures during this experiment: " + str(min_temp) + " - " + str(max_temp) + " C \n The first scan for this experiment occurred at: " + str(initial_time)
  plt.gcf().text(0.5, -0.2, metadata_text, fontsize = 8, horizontalalignment="center")

  save_path = save_path + experiment_name + "_" + "_".join(scan_names) + ".png"
  if save_or_not: plt.savefig(save_path, bbox_inches='tight')




def all_data_plotter(df, wavelengths, experiment_name, save_or_not, save_path):
  plot_prepper(wavelengths, "All Data")

  data = df
  list_of_scans = []
  list_of_temperatures = []
  number_of_scans = data.shape[0]
  colors = list(cm.rainbow(np.linspace(0, 1, number_of_scans)))
  initial_time = datetime.strptime(data.iloc[1,data.columns.get_loc("time")], "%Y-%m-%d %H:%M:%S")
  #initial_time = "x"

  for i in range(1,number_of_scans):
    list_of_scans = list_of_scans + [str(data.index.tolist()[i][0]) + " " + str(data.index.tolist()[i][1])]
    list_of_temperatures = list_of_temperatures + [data.iloc[i,data.columns.get_loc("temp")]]
    x_data = data.iloc[data.index.get_loc("Wavelength"),df.columns.get_loc("time")+1:].transpose()
    y_data = data.iloc[i,df.columns.get_loc("time")+1:]

    plt.plot(x_data, y_data,color=tuple(colors[i-1]))

  patches = [mpatches.Patch(color=color, label=list_of_scans) for list_of_scans, color in zip(list_of_scans, colors)]
  plt.legend(patches, list_of_scans, loc='upper right', frameon=False,prop={'size':4})

  max_temp = max(list_of_temperatures)
  min_temp = min(list_of_temperatures)
  metadata_text = experiment_name + "\n" + "Range of temperatures during this experiment: " + str(min_temp) + " - " + str(max_temp) + " C \n The first scan for this experiment occurred at: " + str(initial_time)
  plt.gcf().text(0.5, -0.2, metadata_text, fontsize = 8, horizontalalignment="center")
  save_path = save_path + experiment_name + "_all_data.png"
  if save_or_not: plt.savefig(save_path, bbox_inches='tight')





def abs_vs_time_plotter(df, wavelengths, well_names, wavelength_to_plot, wells, experiment_name, save_or_not, save_path):
  
  for i in well_names:
    if i not in wells: raise Exception("The provided well (" + str(i) + ") was never scanned so it cannot be plotted")
  if wavelength_to_plot not in wavelengths: raise Exception("The provided wavelength to plot (" + str(wavelength_to_plot) + ") was not in the scannned range so it cannot be plotted")

  title = ", ".join(well_names) + " at " + str(wavelength_to_plot) + " nm"
  plot_prepper2_part1(title)

  number_of_lines = len(well_names)
  list_of_temperatures = []
  colors = list(cm.rainbow(np.linspace(0, 1, number_of_lines)))

  for wells in well_names:
    data = df.loc[pd.IndexSlice[:, ["Wavelength", wells], :]]
    initial_time = datetime.strptime(data.iloc[1,data.columns.get_loc("time")], "%Y-%m-%d %H:%M:%S")
    list_of_time_floats = list(data["Time_Since_Last_Scan"])[1:]
    abs_data = list(data[wavelength_to_plot])[1:]
    list_of_temperatures = list(data["temp"])[1:]
    total_time = max(list_of_time_floats)

    x_data = list_of_time_floats
    y_data = abs_data
    plt.plot(x_data, y_data,linestyle='--', marker='o', color=tuple(colors[well_names.index(wells)]))

  plot_prepper2_part2(total_time, y_min = 0, y_max = 1, y_title = "Absorbance (a.u.)")
  patches = [mpatches.Patch(color=color, label=well_names) for well_names, color in zip(well_names, colors)]
  if len(well_names) != 1:
    plt.legend(patches, well_names, loc='upper right', frameon=False,prop={'size':8})

  max_temp = max(list_of_temperatures)
  min_temp = min(list_of_temperatures)
  metadata_text = experiment_name + "\n" + "Range of temperatures during this experiment: " + str(min_temp) + " - " + str(max_temp) + " C \n The first scan for this experiment occurred at: " + str(initial_time)
  plt.gcf().text(0.5, -0.2, metadata_text, fontsize = 8, horizontalalignment="center")

  save_path = save_path + experiment_name + "_" + title + ".png"
  if save_or_not: plt.savefig(save_path, bbox_inches='tight')





def lambda_max_plotter(df, wavelengths, well_names, wavelength_range, wells, experiment_name, save_or_not, save_path):

  for i in well_names:
    if i not in wells: raise Exception("The provided well (" + str(i) + ") was never scanned so it cannot be plotted")
  for j in (wavelength_range):
    if j not in wavelengths: raise Exception("The provided limit to the wavelength range (" + str(j) + ") was not in the scannned range so it cannot be plotted")

  #makes the absorbance plot
  title = ", ".join(well_names) + " Max in Range " + str(wavelength_range) + " Absorbance Plot"
  plot_prepper2_part1(title)

  number_of_lines = len(well_names)
  list_of_temperatures = []
  colors = list(cm.rainbow(np.linspace(0, 1, number_of_lines)))

  for wells in well_names:
    data = df.loc[pd.IndexSlice[:, ["Wavelength", wells], :]]
    initial_time = datetime.strptime(data.iloc[1,data.columns.get_loc("time")], "%Y-%m-%d %H:%M:%S")
    list_of_time_floats = list(data["Time_Since_Last_Scan"])[1:]
    absorbance_data = data.loc[:, wavelength_range[0]:wavelength_range[1]]
    lambda_max_wavelengths = absorbance_data.idxmax(axis = 1).tolist()[1:]
    absorbance_max = absorbance_data.max(axis = 1).tolist()[1:]
    list_of_temperatures = list(data["temp"])[1:]
    total_time = max(list_of_time_floats)

    x_data = list_of_time_floats
    y_1_data = lambda_max_wavelengths
    y_2_data = absorbance_max
    plt.plot(x_data, y_2_data,linestyle='--', marker='o', color=tuple(colors[well_names.index(wells)]))

  plot_prepper2_part2(total_time, y_min = 0, y_max = 1, y_title = "Absorbance (a.u.)")
  patches = [mpatches.Patch(color=color, label=well_names) for well_names, color in zip(well_names, colors)]
  if len(well_names) != 1:
    plt.legend(patches, well_names, loc='upper right', frameon=False,prop={'size':8})

  max_temp = max(list_of_temperatures)
  min_temp = min(list_of_temperatures)
  metadata_text = experiment_name + "\n" + "Range of temperatures during this experiment: " + str(min_temp) + " - " + str(max_temp) + " C \n The first scan for this experiment occurred at: " + str(initial_time)
  plt.gcf().text(0.5, -0.2, metadata_text, fontsize = 8, horizontalalignment="center")

  save_path = save_path + experiment_name + "_" + title + ".png"
  if save_or_not: plt.savefig(save_path, bbox_inches='tight')


  #makes the wavelength plot
  title = ", ".join(well_names) + " Max in Range " + str(wavelength_range) + " Wavelength Plot"
  plot_prepper2_part1(title)

  number_of_lines = len(well_names)
  list_of_temperatures = []
  colors = list(cm.rainbow(np.linspace(0, 1, number_of_lines)))

  for wells in well_names:
    data = df.loc[pd.IndexSlice[:, ["Wavelength", wells], :]]
    initial_time = datetime.strptime(data.iloc[1,data.columns.get_loc("time")], "%Y-%m-%d %H:%M:%S")
    list_of_time_floats = list(data["Time_Since_Last_Scan"])[1:]
    absorbance_data = data.loc[:, wavelength_range[0]:wavelength_range[1]]
    lambda_max_wavelengths = absorbance_data.idxmax(axis = 1).tolist()[1:]
    absorbance_max = absorbance_data.max(axis = 1).tolist()[1:]
    list_of_temperatures = list(data["temp"])[1:]
    total_time = max(list_of_time_floats)

    x_data = list_of_time_floats
    y_1_data = lambda_max_wavelengths
    y_2_data = absorbance_max
    plt.plot(x_data, y_1_data,linestyle='--', marker='o', color=tuple(colors[well_names.index(wells)]))

  plot_prepper2_part2(total_time, y_min = wavelength_range[0], y_max = wavelength_range[1], y_title = "Wavelength (nm)")
  patches = [mpatches.Patch(color=color, label=well_names) for well_names, color in zip(well_names, colors)]
  if len(well_names) != 1:
    plt.legend(patches, well_names, loc='upper right', frameon=False,prop={'size':8})

  max_temp = max(list_of_temperatures)
  min_temp = min(list_of_temperatures)
  metadata_text = experiment_name + "\n" + "Range of temperatures during this experiment: " + str(min_temp) + " - " + str(max_temp) + " C \n The first scan for this experiment occurred at: " + str(initial_time)
  plt.gcf().text(0.5, -0.2, metadata_text, fontsize = 8, horizontalalignment="center")

  save_path = save_path + experiment_name + "_" + title + ".png"
  if save_or_not: plt.savefig(save_path, bbox_inches='tight')


def lambda_max_output(df, wavelengths, well_names, wavelength_range, wells, experiment_name, save_or_not, save_path):

  for i in well_names:
    if i not in wells: raise Exception("The provided well (" + str(i) + ") was never scanned so it cannot be plotted")
  for j in (wavelength_range):
    if j not in wavelengths: raise Exception("The provided limit to the wavelength range (" + str(j) + ") was not in the scannned range so it cannot be plotted")

  title = ", ".join(well_names) + " Max in Range " + str(wavelength_range) + " Data"
  save_path = save_path + experiment_name + "_" + title + ".csv"
  if save_or_not:
    csv_doc = open(save_path, 'w', encoding='UTF8', newline='')
    writer = csv.writer(csv_doc)
  number_of_lines = len(well_names)

  for wells in well_names:
    data = df.loc[pd.IndexSlice[:, ["Wavelength", wells], :]]
    initial_time = datetime.strptime(data.iloc[1,data.columns.get_loc("time")], "%Y-%m-%d %H:%M:%S")
    list_of_time_floats = list(data["Time_Since_Last_Scan"])[1:]
    absorbance_data = data.loc[:, wavelength_range[0]:wavelength_range[1]]
    lambda_max_wavelengths = absorbance_data.idxmax(axis = 1).tolist()[1:]
    absorbance_max = absorbance_data.max(axis = 1).tolist()[1:]
    total_time = max(list_of_time_floats)

    chemical_names = data.iloc[0,data.columns.get_loc("original_scan_id")+1:data.columns.get_loc("time of last reagent added")].to_list()
    chemical_concentrations = data.iloc[-1,data.columns.get_loc("original_scan_id")+1:data.columns.get_loc("time of last reagent added")].to_list()
    row_1 = [wells] + chemical_names + ["time (min): "] + list_of_time_floats
    row_2 = [wells] + chemical_concentrations + ["lambda max wavelength (nm): "] + lambda_max_wavelengths
    row_3 = [wells] + chemical_concentrations + ["absorbance at lambda max (a.u.): "] + absorbance_max


    print(wells)
    print(row_1)
    print(row_2)
    print(row_3)
    print(" ")

    if save_or_not:
      writer.writerow(row_1)
      writer.writerow(row_2)
      writer.writerow(row_3)
      writer.writerow([])

  if save_or_not:
    csv_doc.close()
    print("data saved to csv")





def kinetics_summary_plotter(df, wavelengths, wells, experiment_name, save_or_not, save_path, presentation_mode):
  fig, axs = plt.subplots(8, 12, dpi=300, figsize=(60, 40), subplot_kw=dict(sharex = True,sharey = True))
  plt.subplots_adjust(wspace=0.3, hspace= 0.3)

  for i in ["A","B","C","D","E","F","G","H"]:
    for j in range(0,12):
      col_num = int(j)
      row_num = ord(i)-65
      axs[row_num, col_num].set_title(i + str(j+1))
      axs[row_num, col_num].set_xlabel("Wavelength (nm)")
      axs[row_num, col_num].set_ylabel("Absorbance (a.u.)")
      if presentation_mode:
        axs[row_num, col_num].tick_params(which='both',bottom='off',left='off',right='off',top='off')
        axs[row_num, col_num].label_outer()
        #axs[row_num, col_num].set_linewidth(2)
        axs[row_num, col_num].spines['top'].set_linewidth(2)
        axs[row_num, col_num].spines['right'].set_linewidth(2)
        axs[row_num, col_num].spines['bottom'].set_linewidth(2)
        axs[row_num, col_num].spines['left'].set_linewidth(2)



  for i in wells:
    list_of_time_strings = []
    data = df.loc[pd.IndexSlice[:, ["Wavelength"] + [i]], :]
    number_of_scans = data.shape[0]
    colors = list(cm.rainbow(np.linspace(0, 1, number_of_scans)))
  
    #initial_time = datetime.strptime(data.iloc[1,data.columns.get_loc("time")], "%Y-%m-%d %H:%M:%S")
    #list_of_time_floats = list(data["Time_Since_Last_Scan"])[1:]
    #list_of_temperatures = list(data["temp"])[1:]

    col_num = int(i[1:])-1
    row_num = ord(i[0])-65
    axs[row_num, col_num].set_title(i[0]+str(col_num+1))

    for i in range(1,number_of_scans):
      #creates string labels from the list of times (adds the well name if multiple wells are on the same plot)
      #list_of_time_strings = list_of_time_strings + [str(list_of_time_floats[i-1]) + " minutes"]
      
      x_data = data.iloc[data.index.get_loc("Wavelength"),df.columns.get_loc("time")+1:].transpose()
      y_data = data.iloc[i,df.columns.get_loc("time")+1:]

      if not presentation_mode: axs[row_num, col_num].plot(x_data, y_data,color=tuple(colors[i-1]))
      else:axs[row_num, col_num].plot(x_data, y_data,color=tuple(colors[i-1]), linewidth=2)
      axs[row_num, col_num].set_xlabel("Wavelength (nm)")
      axs[row_num, col_num].set_ylabel("Absorbance (a.u.)")
      if presentation_mode:
        axs[row_num, col_num].tick_params(which='both',bottom='off',left='off',right='off',top='off')
        axs[row_num, col_num].label_outer()
      

    #patches = [mpatches.Patch(color=color, label=list_of_time_strings) for list_of_time_strings, color in zip(list_of_time_strings, colors)]
    #plt.legend(patches, list_of_time_strings, loc='upper right', frameon=False,prop={'size':8})
  





  """  

  for ax in axs.flat:
      ax.set(xlabel='x-label', ylabel='y-label')

  # Hide x labels and tick labels for top plots and y ticks for right plots.
  for ax in axs.flat:
      ax.label_outer()"""

  """    def plot_kin_subplots(self,df,n_cycles,wells,filename=None):
        '''
        TODO this function doesn't save properly, but it does show. Don't know issue  
        plots kinetics for each well in the order given by wells.  
        params:  
            df df: the scan data  
            int n_cycles: the number of cycles for the scan data  
            list<str> wells: the wells you want to plot in order
        Postconditions:  
            plot has been written with name "{filename}_overlay.png" to the plotting dir.  
            If filename is not supplied, name is kin_subplots
        '''
        if not filename:
            filename=kin_subplots
        x_vals = list(range(300,1001))
        colors = list(cm.rainbow(np.linspace(0, 1, n_cycles)))
        fig, axes = plt.subplots(8, 12, dpi=300, figsize=(50, 50),subplot_kw=dict(box_aspect=1,sharex = True,sharey = True))
        for idx, (chem_name, ax) in enumerate(zip(wells, axes.flatten())):
            ax.set_title(chem_name)
            self._plot_kin(ax, df, n_cycles, chem_name)
            plt.subplots_adjust(wspace=0.3, hspace= -0.1)
        
            ax.tick_params(
                which='both',
                bottom='off',
                left='off',
                right='off',
                top='off'
            )
            ax.set_xlim((300,1000))
            ax.set_ylim((0,1.0))
            ax.set_xlabel("Wavlength (nm)")
            ax.set_ylabel("Absorbance (A.U.)")
            ax.set_xticks(range(301, 1100, 100))
            #ax.set_aspect(adjustable='box')
            #ax.set_yticks(range(0,1))
        else:
            [ax.set_visible(False) for ax in axes.flatten()[idx+1:]]
        plt.savefig(os.path.join(self.plot_path, '{}.png'.format(filename)))
        plt.close()"""











#Change the below paths to match your environment.

def run_plotters(run_kinetics_plotter, run_scan_plotter, run_all_data_plotter, run_abs_vs_time_plotter, run_kinetics_summary_plotter, experiment_name, well_names, scan_names, wavelength_to_plot, wavelenth_range, subtract_blanks_or_not, blank_well, save_or_not, initial_time_as_last_reagent_added, presentation_mode):

  load_path = "/home/void/school/capstone/OT2Control/TestingDatasets/pr_data/" + experiment_name + "_full.csv"
  save_path = "/home/void/school/capstone/OT2Control/TestingDatasets/plots/" + experiment_name + "plot"

  if isinstance(well_names, str) or isinstance(well_names, list): well_names = [well_names]
  if isinstance(well_names, tuple): well_names = list(well_names)
  if isinstance(scan_names, str) or isinstance(scan_names, list): scan_names = [scan_names]
  if isinstance(scan_names, tuple): scan_names = list(scan_names)
  if isinstance(wavelength_to_plot, tuple): wavelength_to_plot = list(wavelength_to_plot)
  if isinstance(wavelength_to_plot, int): wavelength_to_plot = [wavelength_to_plot]

  df, wavelengths, scans, wells =load_df(load_path, subtract_blanks_or_not, blank_well, initial_time_as_last_reagent_added)
  
  if run_kinetics_plotter:
    for i in well_names:
      if not isinstance(i, list): i = [i]
      kinetics_plotter(df,wavelengths, i, wells, experiment_name, save_or_not, save_path)
  if run_scan_plotter:
    for i in scan_names:
      if not isinstance(i, list): i = [i]
      scan_plotter(df,wavelengths, i, scans, experiment_name, save_or_not, save_path)
  if run_all_data_plotter:
    all_data_plotter(df,wavelengths, experiment_name, save_or_not, save_path)
  if run_abs_vs_time_plotter:
    for i in well_names:
      for j in wavelength_to_plot:
        if not isinstance(i, list): i = [i]
        abs_vs_time_plotter(df,wavelengths, i, j, wells, experiment_name, save_or_not, save_path)
  if run_lambda_max_plotter:
    for i in well_names:
        if not isinstance(i, list): i = [i]
        lambda_max_plotter(df,wavelengths, i, wavelength_range, wells, experiment_name, save_or_not, save_path)
  if run_lambda_max_output:
    for i in well_names:
        if not isinstance(i, list): i = [i]
        lambda_max_output(df,wavelengths, i, wavelength_range, wells, experiment_name, save_or_not, save_path)        
  if run_specific_wells_scans_plotter:
      for i in well_names:
        for j in scan_names:
          if not isinstance(i, list): i = [i]
          if not isinstance(j, list): j = [j]
          specific_wells_scans_plotter(df,wavelengths, i, j, experiment_name, save_or_not, save_path)
  if run_kinetics_summary_plotter:
    kinetics_summary_plotter(df, wavelengths, wells, experiment_name, save_or_not, save_path, presentation_mode)
  print(df[0:10])





run_plotters(run_kinetics_plotter, run_scan_plotter, run_all_data_plotter, run_abs_vs_time_plotter, run_kinetics_summary_plotter, experiment_name, well_names, scan_names, wavelength_to_plot, wavelength_range, subtract_blanks_or_not, blank_well, save_or_not, initial_time_as_last_reagent_added, presentation_mode)



/tmp/ipykernel_1726/2791534671.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["scan id"][0]="Wavelength"
/tmp/ipykernel_1726/2791534671.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["well"][0]="Wavelength"


['scan_000', 'scan_001', 'scan_002', 'scan_003', 'scan_004', 'scan_005', 'scan_006', 'scan_007', 'scan_008', 'scan_009', 'scan_010', 'scan_011', 'scan_012', 'scan_013', 'scan_014', 'scan_015', 'scan_016', 'scan_017', 'scan_018', 'scan_019', 'scan_020', 'scan_021', 'scan_022', 'scan_023', 'scan_024']
A02
['A02', 'well name', 'hydrogen_peroxide', 'e_ctab', 'ctab', 'silver_nitrate', 'ectab', 'sodium_borohydride', 'trisodium_citrate', 'time (min): ', 1.706226, 32.406226, 36.98955933333333, 41.57289266666667, 46.156226, 50.73955933333333]
['A02', 'CNH_040__rxn8C1.0', 12.5, 0.0, 0.0, 0.09375, 0.015, 0.625, 1.25, 'lambda max wavelength (nm): ', 401, 950, 948, 950, 949, 950]
['A02', 'CNH_040__rxn8C1.0', 12.5, 0.0, 0.0, 0.09375, 0.015, 0.625, 1.25, 'absorbance at lambda max (a.u.): ', 0.205, 0.4600000000000001, 0.44100000000000006, 0.43200000000000005, 0.42100000000000004, 0.41200000000000003]
 
B02
['B02', 'well name', 'hydrogen_peroxide', 'e_ctab', 'ctab', 'silver_nitrate', 'ectab', 'sodium_b

In [4]:
def data_checker(path, well_names, scan_names, subtract_blanks_or_not, blank_well,
                 initial_time_as_last_reagent_added):

  if isinstance(well_names, str) or isinstance(well_names, list): well_names = [well_names]
  if isinstance(well_names, tuple): well_names = list(well_names)
  if isinstance(scan_names, str) or isinstance(scan_names, list): scan_names = [scan_names]
  if isinstance(scan_names, tuple): scan_names = list(scan_names)

  df, wavelengths, scans, wells =load_df(path, subtract_blanks_or_not, blank_well, initial_time_as_last_reagent_added)

  print(df.head())
  print(type(df))
  
data_checker("/home/void/school/capstone/OT2Control/TestingDatasets/pr_data/CNH_040_full.csv", well_names, scan_names, subtract_blanks_or_not, blank_well, initial_time_as_last_reagent_added)

/tmp/ipykernel_1726/2791534671.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["scan id"][0]="Wavelength"
/tmp/ipykernel_1726/2791534671.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["well"][0]="Wavelength"


['scan_000', 'scan_001', 'scan_002', 'scan_003', 'scan_004', 'scan_005', 'scan_006', 'scan_007', 'scan_008', 'scan_009', 'scan_010', 'scan_011', 'scan_012', 'scan_013', 'scan_014', 'scan_015', 'scan_016', 'scan_017', 'scan_018', 'scan_019', 'scan_020', 'scan_021', 'scan_022', 'scan_023', 'scan_024']
                       Scan_Number original_scan_id          well name  \
scan id    well                                                          
Wavelength Wavelength  Scan_Number          scan id          well name   
scan_000   C01                   1     CNH_040_t0-b  CNH_040__rxn2C1.0   
scan_001   D01                   2     CNH_040_t0-c  CNH_040__rxn3C1.0   
scan_002   E01                   3     CNH_040_t0-d  CNH_040__rxn4C1.0   
scan_003   F01                   4     CNH_040_t0-e  CNH_040__rxn5C1.0   

                       hydrogen_peroxide  e_ctab    ctab  silver_nitrate  \
scan id    well                                                            
Wavelength Wavelength  hydro

In [6]:
data_checker("/home/void/school/capstone/OT2Control/TestingDatasets/pr_data/CNH_039_full.csv", well_names, scan_names, subtract_blanks_or_not, "A05", initial_time_as_last_reagent_added)

/tmp/ipykernel_1726/2791534671.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["scan id"][0]="Wavelength"
/tmp/ipykernel_1726/2791534671.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["well"][0]="Wavelength"


['scan_000', 'scan_001', 'scan_002', 'scan_003', 'scan_004', 'scan_005', 'scan_006', 'scan_007', 'scan_008', 'scan_009', 'scan_010', 'scan_011', 'scan_012', 'scan_013', 'scan_014', 'scan_015', 'scan_016', 'scan_017', 'scan_018', 'scan_019', 'scan_020', 'scan_021', 'scan_022', 'scan_023', 'scan_024', 'scan_025', 'scan_026', 'scan_027', 'scan_028', 'scan_029', 'scan_030', 'scan_031', 'scan_032', 'scan_033', 'scan_034', 'scan_035', 'scan_036', 'scan_037', 'scan_038', 'scan_039']
                       Scan_Number original_scan_id          well name  \
scan id    well                                                          
Wavelength Wavelength  Scan_Number          scan id          well name   
scan_000   C05                   1     CNH_039_t0-b  CNH_039__rxn2C1.0   
scan_001   D05                   2     CNH_039_t0-c  CNH_039__rxn3C1.0   
scan_002   E05                   3     CNH_039_t0-d  CNH_039__rxn4C1.0   
scan_003   F05                   4     CNH_039_t0-e  CNH_039__rxn5C1.0   



In [ ]:
have to 

In [ ]:
#Concatenate datasets into a large dataframe
#we will use only CNH_0XX_full.csv files except for scan CNH_013
#We can get a blank well row and append it to all of these to use it
CNH08 = "pr_data/CNH_008_full.csv"
CNH13 = "pr_data/CNH_013_full_df.csv"
CNH16 = "pr_data/CNH_016_full.csv"
CNH17 = "pr_data/CNH_017_full.csv"
CNH33 = "pr_data/CNH_033_full.csv"
CNH39 = "pr_data/CNH_039_full.csv"
CNH40 = "pr_data/CNH_040_full.csv"
CNH08 = load_df(CNH08, True, "", True)